In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

Exemple sur stackoverflow

```python
sub_qs = Building.objects.filter(mpoly__intersects=OuterRef("mpoly"))
sub_qs = sub_qs.annotate(
    intersection=Intersection("mpoly", OuterRef("mpoly")),
    intersection_area=Area(Transform("intersection", 2154)),
)
sub_qs = sub_qs.aggregate(Sum("intersection_area"))

Area.objects.all().annotate(
    surface_built=Cast(
        Subquery(sub_qs[:1]),
        DecimalField(max_digits=3, decimal_places=2)
    )
).update(density=F("surface_built")/F("surface"))
```

In [13]:
from public_data.models import Ocsge, ZoneConstruite
from django.contrib.gis.db.models import MultiPolygonField
from django.contrib.gis.db.models.functions import Area, Transform, Intersection
from django.db.models import DecimalField, FloatField, Sum, F, Subquery, OuterRef
from django.db.models.functions import Cast




(
    ZoneConstruite.objects.filter(year=2016)
    .annotate(building_surface=Subquery(
        (
            Ocsge.objects.filter(year=2016)
            .annotate(outer_mpoly=Intersection("mpoly", OuterRef("mpoly")))
            .annotate(building_surface=Area(Transform("outer_mpoly", 2154)))
            .values("building_surface")
        )[:1]
    ))
    .update(built_density=Cast(
        F("building_surface")/F("surface"),
        FloatField(),
    ))
)


AttributeError: 'ResolvedOuterRef' object has no attribute '_output_field_or_none'